In [1]:
#import librarys
import numpy as np
import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from ipynb.fs.full.nltk_utils1 import *
from ipynb.fs.full.modele import NeuralNet

['my', 'name', 'is', 'oumaima']
my name is oumaima organize?


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nlout\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
with open('pairs.json', 'r') as f:
    intents = json.load(f)

In [3]:
all_words = []
tags = []
xy = []

In [4]:
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

In [5]:
# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

81 patterns
18 tags: ['funny', 'goodbye', 'greeting', 'item1', 'item2', 'item3', 'item4', 'menu', 'moody', 'name', 'positive', 'thanks', 'tips+', 'tipsA', 'tipsB', 'tipsC', 'tipsD', 'tipsE']
88 unique stemmed words: ["'", "'s", '(', ')', '*', '+', '1', '2', '3', '4', 'a', 'alert', 'amaz', 'amus', 'anyon', 'appreci', 'are', 'awe', 'awesom', 'b', 'bonjour', 'bye', 'c', 'cesd', 'confid', 'content', 'cool', 'cool-moodi', 'curiou', 'd', 'day', 'depress', 'do', 'e', 'elev', 'encourag', 'energ', 'funlov', 'funni', 'glad', 'good', 'goodby', 'grate', 'great', 'greet', 'hello', 'help', 'hey', 'hi', 'hola', 'hope', 'how', 'ill', 'inspir', 'interest', 'is', 'joke', 'joy', 'know', 'later', 'lot', 'love', 'me', 'menu', 'moodi', 'my', 'name', 'ok', 'okay', 'optimist', 'peac', 'r', 'satisfact', 'see', 'seren', 'silli', 'someth', 'swl', 'tell', 'thank', 'that', 'there', 'trust', 'uplift', 'welcom', 'wellb', 'wonder', 'you']


In [6]:
# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)


In [7]:
# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

88 18


In [8]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples


In [9]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)





In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [11]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device=device,dtype=torch.int64)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [100/1000], Loss: 0.1442
Epoch [200/1000], Loss: 2.6905
Epoch [300/1000], Loss: 0.0004
Epoch [400/1000], Loss: 0.0349
Epoch [500/1000], Loss: 0.0358
Epoch [600/1000], Loss: 0.0000
Epoch [700/1000], Loss: 0.0118
Epoch [800/1000], Loss: 0.0019
Epoch [900/1000], Loss: 0.0000
Epoch [1000/1000], Loss: 0.0002


In [13]:
print(f'final loss: {loss.item():.4f}')

final loss: 0.0002


In [14]:

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth
